In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Rescaling
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [13]:
BATCH_SIZE = 8
IMG_HEIGHT = 224
IMG_WIDTH = 224
DATA_DIR = 'data'
EPOCHS = 10

In [14]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 6632 files belonging to 3 classes.
Using 5969 files for training.
Found 6632 files belonging to 3 classes.
Using 663 files for validation.


In [15]:
class_names = train_ds.class_names
print(class_names)

['Igneous', 'Metamorphic', 'Sedimentary']


In [16]:
resnet50 = tf.keras.applications.resnet_v2.ResNet50V2(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

In [17]:
model = keras.Sequential([
    Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    resnet50,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(256, activation='relu'),
    Dense(3, activation='softmax')
])

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_3 (Dense)             (None, 1024)              102761472 
                                                                 
 dense_4 (Dense)             (None, 256)               262400    
                                                                 
 dense_5 (Dense)             (None, 3)                 771       
                                                                 
Total params: 126,589,443
Trainable params: 126,544,00

In [19]:
model.compile(optimizer='rmsprop', loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [20]:
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

Epoch 1/10


2022-01-28 13:35:34.795761: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


104/747 [===>..........................] - ETA: 8:33 - loss: 3.6750 - accuracy: 0.3474

KeyboardInterrupt: 